In [8]:
trait ListIF[L,A] {
    def empty: L
    def head(l: L): Option[A]
    def tail(l: L): L
    def cons(a: A, I: L): L
    def append(l1: L, l2: L): L
//     def map[A,B](f: A=>B)(l: L[A]): L[B]
}

defined trait ListIF

In [9]:
trait ListIF[L[_]] {
    def empty[A]: L[A]
    def head[A](l: L[A]): Option[A]
    def tail[A](l: L[A]): L[A]
    def cons[A](a: A, l: L[A]): L[A]
    def append[A](l1: L[A], l2: L[A]): L[A]
    def map[A,B](f: A=>B)(l: L[A]): L[B]  // added
}

defined trait ListIF

In [12]:
abstract class Iter[I[_]] {
    def getValue[A](a: I[A]): Option[A]
    def getNext[A](a: I[A]): I[A]
}
abstract class Iterable[R[_]] {
    type Itr[_]
    def iter[A](a: R[A]): Itr[A]
    def iterIF: Iter[Itr]
}
def printElements[I[_],A](xs: I[A])(implicit IT: Iter[I]): Any = {
    IT.getValue(xs) match {
        case None => 0
        case Some(a) => {
            println(a)
            printElements(IT.getNext(xs))
        }
    }
}

defined class Iter
defined class Iterable
defined function printElements

In [22]:
def testList[L[_]](implicit LI: ListIF[L], IT: Iter[L]) = {
    val l1 = LI.cons(3.3, LI.cons(2.2, LI.cons(1.5, LI.empty)))
    val l2 = LI.map((n:Double)=>n.toInt)(l1)
    val l3 = LI.map((n:Int)=>n.toString+".xx")(l2)
    printElements(l3)
}

defined function testList

In [23]:
implicit def listIter: Iter[List] = {
    new Iter[List] {
        def getValue[A](a: List[A]) = a.headOption
        def getNext[A](a: List[A]) = a.tail
    }
}
implicit val listIF: ListIF[List] = {
    new ListIF[List] {
        def empty[A] = Nil
        def head[A](l: List[A]) = l.headOption
        def tail[A](l: List[A]) = l.tail
        def cons[A](a: A, l: List[A]) = a::l
        def append[A](l1: List[A], l2: List[A]) = l1:::l2
        def map[A,B](f: A=>B)(l: List[A]) = l.map(f)
    }
}
testList[List]

3.xx
2.xx
1.xx


defined function listIter
listIF: ListIF[List] = ammonite.$sess.cmd22$Helper$$anon$2@36f8f6e6
res22_2: Any = 0

In [24]:
trait Iter[I[_]] {
    def getValue[A](a: I[A]): Option[A]
    def getNext[A](a: I[A]) : I[A]
}
trait Foo[I[_[_]]] {
    def get: I[List]
}
def f(x: Foo[Iter]) : Iter[List] = x.get

defined trait Iter
defined trait Foo
defined function f

---
### Type class to OO class

In [26]:
// OOP style
trait DataProcessor {
    def input(s: String): DataProcessor
    def output(): String
}
trait DPFactory {
    def getTypes: List[String]
    def makeDP(dptype: String): DataProcessor
}

defined trait DataProcessor
defined trait DPFactory

In [29]:
trait DataProcessor[D] {
    def input(d: D, s: String): D
    def output(d: D): String
}
trait DataProcessorBox {
    type DP
    val data: DP
    val interface: DataProcessor[DP]
}
trait DPFactory {
    def getTypes: List[String]
    def makeDP(dptype: String): DataProcessorBox
}

defined trait DataProcessor
defined trait DataProcessorBox
defined trait DPFactory

In [56]:
import scala.language.higherKinds
import scala.language.implicitConversions

trait DataProcessor[D] {
    def input(d: D, s: String): D
    def output(d: D): String
}
trait Box[S[_]] {
    type Data
    val data: Data
    val interface: S[Data]
}
trait DPFactory {
    def getTypes: List[String]
    def makeDP(dptype: String): Box[DataProcessor]
}

import scala.language.higherKinds

import scala.language.implicitConversions


defined trait DataProcessor
defined trait Box
defined trait DPFactory

In [71]:
object Box {
    implicit def apply[D,S[_]](d: D)(implicit i: S[D]): Box[S] = {
        new Box[S] {
            type Data = D
            val data = d
            val interface = i
        }
    }
}

defined object Box

In [78]:
trait UserInteraction {
    def run(factory: DPFactory): Unit
}

defined trait UserInteraction

In [79]:
val userInteraction = new UserInteraction {
    def run(factory: DPFactory) = {
        val dptype = scala.io.StdIn.readLine("Input a processor type " + factory.getTypes.toString + ": ")
        val dp = factory.makeDP(dptype)
        val d_done = getInputs(dp.data)(dp.interface)
        printOutputs(d_done)(dp.interface)
    }
    def getInputs[D](d: D)(implicit DP: DataProcessor[D]): D = {
        val d2 = DP.input(d, scala.io.StdIn.readLine("Input Data: "))
        val done = scala.io.StdIn.readLine("More inputs? [Y/N] ")
        if (done.toLowerCase() == "n") d2
        else getInputs(d2)
    }
    def printOutputs[D](d: D)(implicit DP: DataProcessor[D]) = {
        println("The result of processing your inputs is: ")
        println(DP.output(d))
    }
}

userInteraction: AnyRef with UserInteraction{def getInputs[D](d: D)(implicit DP: cmd78.this.cmd55.DataProcessor[D]): D;def printOutputs[D](d: D)(implicit DP: cmd78.this.cmd55.DataProcessor[D]): Unit} = ammonite.$sess.cmd78$Helper$$anon$1@28eb76c5

In [82]:
val dpfactory = new DPFactory {
    def getTypes = List("sum", "mult")
    def makeDP(dptype: String) = {
        if (dptype == "sum") makeProc(0, (x,y) => x+y)
        else makeProc(1, (x,y) => x*y)
    }
    def makeProc(init: Int, op: (Int,Int)=>Int): Box[DataProcessor] = {
        implicit val dp = new DataProcessor[Int] {
            def input(d: Int, s: String) = op(d, s.toInt)
            def output(d: Int) = d.toString()
        }
//         init
        Box.apply[Int,DataProcessor](init)(dp)
    }
}

dpfactory: AnyRef with DPFactory{def makeProc(init: Int, op: (Int, Int) => Int): cmd81.this.cmd55.Box[cmd81.this.cmd55.DataProcessor]} = ammonite.$sess.cmd81$Helper$$anon$1@29f0339f

In [84]:
userInteraction.run(dpfactory)

Input a processor type List(sum, mult): 

 mult


Input Data: 

 5


More inputs? [Y/N] 

...

Input Data: 

 100


More inputs? [Y/N] 

 n


The result of processing your inputs is: 
12000
